## MLJAR

### Instalacja

https://github.com/mljar/mljar-supervised

In [ ]:
# !pip install mljar-supervised

### Przykład wywołania MLJAR

Dokumentacja pakietu: https://supervised.mljar.com/

In [ ]:


import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from supervised.automl import AutoML # mljar-supervised

print('########## LOADING DATA ##########')
# Load the data
housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(housing.data, columns=housing.feature_names),
    housing.target,
    test_size=0.25,
    random_state=123,
)

print('########## TRAINING MODELS ##########')
# train models with AutoML
automl = AutoML()
automl.fit(X_train, y_train)


print('########## PREDICTIONS ##########')
# compute the MSE on test data
predictions = automl.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, predictions))

c:\Users\Lenovo\Documents\Teaching\2025_2026\2025Z-AutoML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 1.175568 trained in 0.89 seconds
2_DecisionTree rmse 0.814045 trained in 24.75 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 0.466835 trained in 21.22 seconds
4_Default_NeuralNetwork rmse 0.576015 trained in 9.21 seconds
5_Default_RandomForest rmse 0.73272 trained in 11.84 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 0.466835 trained in 0.7 seconds
AutoML fit time: 94.88 seconds
AutoML best model: 3_Default_Xgboost
Test MSE: 0.21027419421342936


### Zadanie 1

Wybierz zbiór danych - może być to na przykład zbiór danych, którym pracowałeś/aś podczas laboratorium z `autogluon`. Przygotuj model/e używając pakietu `mljar` i domyślnych ustawień.

Bazując na wiedzy o przygotowanych modelach i ich jakości, porównaj jak poradził sobie pakiet z tym zadaniem. Zwróć szczególną uwagę na raport generowany przez pakiet. Odpowiedz na pytania:

a) jakie typy modeli rozważył framework?

b) jaką miarę wylicza jako bazową?

c) jakie zmienne są najbardziej istotne w modelach?

Bazując na wiedzy jak działa `autogluon`, porównaj te dwa frameworki i odpowiedz na pytania:

d) który uzyskał lepsze rezultaty?

e) czy w obu przypadkach komitet modeli równie dobrze się sprawdził? (tzn. czy komitet modeli jest najlepszym modelem jaki uzyskaliśmy)


### Zadanie 2

- Na podstawie dokumentacji odpowiedz na pytania:
    -  W jakich trybach (*mode*) możemy trenować modele w `mljar` i jakie mają one przeznaczenia?
    -  Jaki jest domyślny tryb?
    - Jaki tryb wykorzystać aby lepiej zbadać stabilność wyników modeli?
- Porównaj wyniki uzyskane różnymi trybami pracy pakietu `mljar` i porównaj je do `autogluona` z resetem `high_quality` i takim samym limitem czasowym. Czy otrzymane wyniki są znacząco różne? Czy wykorzystując podstawowy tryb jesteśmy w stanie uzyskać dobre wyniki?

In [ ]:
# train models with AutoML
automl = AutoML(mode="Perform", total_time_limit = 60)
automl.fit(X_train, y_train)

### Zadanie 3

Zapoznaj się kwestią zmiennych wrażliwych i fairness, unikalną opcją udostępnianą przez pakiet MLJAR na stronie https://mljar.com/blog/fairness-machine-learning/

W pakiecie możemy modyfikować następujące parametry:

- `fairness_metric` - metric which will be used to decide if the model is fair,

- `fairness_threshold` - threshold used in decision about model fairness,

- `privileged_groups` privileged groups used in fairness metrics computation,

- `underprivileged_groups` - underprivileged groups used in fairness metrics computation.


The `fit()` method accepts `sensitive_features`. When sensitive features are passed to AutoML, the best model will be selected among fair models only. In the AutoML reports, additional information about fairness metrics will be added. The MLJAR AutoML supports two methods for bias mitigation:

- Sample Weighting - assigns weights to samples to treat samples equally,
- Smart Grid Search - similar to Sample Weighting, where different weights are checked to optimize fairness metric.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from supervised.automl import AutoML

data = fetch_openml(data_id=1590, as_frame=True)
X = data.data
y = (data.target == ">50K") * 1
sensitive_features = X[["sex"]]

X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(
    X, y, sensitive_features, stratify=y, test_size=0.75, random_state=42
)

automl = AutoML(
    algorithms=[
        "Xgboost"
    ],
    train_ensemble=False,
    fairness_metric="demographic_parity_ratio",
    fairness_threshold=0.8,
    privileged_groups = [{"sex": "Male"}],
    underprivileged_groups = [{"sex": "Female"}],
)

automl.fit(X_train, y_train, sensitive_features=S_train)